In [34]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
import keras
import numpy as np

In [2]:
num_classes = 2

In [26]:
modelo_base = ResNet50(weights = 'imagenet', include_top = False, input_shape=(224,224,3))

94765736/94765736 [==============================] - 10s 0us/step


In [3]:
modelo = tf.keras.Sequential(
    [modelo_base,
     GlobalAveragePooling2D(),
     Dense(1024, activation = 'relu'),
     Dense(num_classes, activation= 'softmax')
    ])

NameError: name 'modelo_base' is not defined

In [ ]:
modelo.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
              loss= 'categorical_crossentropy',
              metrics=['accuracy'])

In [51]:
path = 'D:/Salve_Aqui_Seus_Arquivos/carlos_araujo/TETI/datasets/dc_small/'

In [6]:
def cria_dataframe(caminho):
    arquivos = []
    rotulos = []
    for arquivo in os.listdir(caminho):
        if arquivo.startswith('cat'):
            rotulos.append('cat')
        elif arquivo.startswith('dog'):
            rotulos.append('dog')
        arquivos.append(arquivo)
    return pd.DataFrame({'arquivo': arquivos, 'rotulo': rotulos})

In [7]:
df_dados = cria_dataframe(os.path.join(path, 'train'))

In [8]:
train_df, validation_df = train_test_split (df_dados, test_size = 0.3,
                                            stratify = df_dados['rotulo'])

In [9]:
test_df = cria_dataframe(os.path.join(path, 'test'))

In [10]:
train_datagen = ImageDataGenerator(preprocessing_function= preprocess_input)
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [11]:
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory = os.path.join(path, 'train'),
    x_col = 'arquivo',
    y_col = 'rotulo',
    target_size = (224,224),
    batch_size = 128,
    class_mode = 'categorical'
)

Found 1400 validated image filenames belonging to 2 classes.


In [12]:
validation_generator = train_datagen.flow_from_dataframe(
    validation_df,
    directory = os.path.join(path, 'train'),
    x_col = 'arquivo',
    y_col = 'rotulo',
    target_size = (224,224),
    batch_size = 128,
    class_mode = 'categorical'
)

Found 600 validated image filenames belonging to 2 classes.


In [52]:
test_generator = test_datagen.flow_from_dataframe(
    test_df,
    directory = os.path.join(path, 'test'),
    x_col = 'arquivo',
    y_col = 'rotulo',
    target_size = (224,224),
    batch_size = 128,
    class_mode = 'categorical'
)

Found 1000 validated image filenames belonging to 2 classes.


In [20]:
modelo.fit(
    train_generator,
    steps_per_epoch = train_generator.samples//train_generator.batch_size,
    validation_data = validation_generator,
    validation_steps = validation_generator.samples//validation_generator.batch_size,
    epochs = 10
)

Epoch 1/10
10/10 [==============================] - 513s 51s/step - loss: 1.9660 - accuracy: 0.6918 - val_loss: 4848.6152 - val_accuracy: 0.5039
Epoch 2/10
10/10 [==============================] - 489s 50s/step - loss: 0.2446 - accuracy: 0.9009 - val_loss: 167.2427 - val_accuracy: 0.5098
Epoch 3/10
10/10 [==============================] - 483s 49s/step - loss: 0.1183 - accuracy: 0.9513 - val_loss: 76.7180 - val_accuracy: 0.5137
Epoch 4/10
10/10 [==============================] - 494s 50s/step - loss: 0.0583 - accuracy: 0.9835 - val_loss: 55.3617 - val_accuracy: 0.5000
Epoch 5/10
10/10 [==============================] - 519s 52s/step - loss: 0.0411 - accuracy: 0.9851 - val_loss: 6.2745 - val_accuracy: 0.5273
Epoch 6/10
10/10 [==============================] - 502s 51s/step - loss: 0.0353 - accuracy: 0.9882 - val_loss: 27.9921 - val_accuracy: 0.5000
Epoch 7/10
10/10 [==============================] - 496s 50s/step - loss: 0.0435 - accuracy: 0.9858 - val_loss: 2.8228 - val_accuracy: 0.492

In [1]:
modelo.save('modelo_10_epochs')

NameError: name 'modelo' is not defined

In [15]:
modelo = keras.models.load_model('D:/Salve_Aqui_Seus_Arquivos/carlos_araujo/TETI/modelo_10_epochs')
path3 = 'D:/Salve_Aqui_Seus_Arquivos/carlos_araujo/TETI/transfer_learning_resnet50_12epochs_dcsmall/'
path2 = 'D:/Salve_Aqui_Seus_Arquivos/carlos_araujo/TETI/amostras/'

In [18]:
modelo2 = keras.models.load_model(path+ 'transfer_learning_resnet50_12epochs_dcsmall.keras')
modelo2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d_1   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_2 (Dense)             (None, 1024)              2098176   
                                                                 
 dense_3 (Dense)             (None, 2)                 2050      
                                                                 
Total params: 25,687,938
Trainable params: 2,100,226
Non-trainable params: 23,587,712
_________________________________________________________________


In [19]:
rotulos_classes = list(validation_generator.class_indices.keys())

In [41]:
def classifica(imagem):
    img = load_img(imagem, target_size=(224,224))
    img_ = image.img_to_array(img)
    img_ = np.expand_dims(img_, axis= 0)
    img_ = preprocess_input(img_)
    preds = modelo.predict(img_)
    classe_predita = np.argmax(preds, axis=1)[0]
    rotulo_predito = rotulos_classes[classe_predita]
    print(f"Classe predita: {rotulo_predito} - {preds[0][classe_predita]:.4f}")

In [53]:
img_pred = (path2+ 'gato1.jpg')

In [54]:
classifica(img_pred)

1/1 [==============================] - 0s 165ms/step
Classe predita: cat - 0.9974


In [55]:
test_loss, test_accuracy = modelo2.evaluate(test_generator, steps= test_generator.samples // test_generator.batch_size)
print(f'Acurácia no teste:{test_accuracy}')

7/7 [==============================] - 81s 11s/step - loss: 0.0792 - accuracy: 0.9821
Acurácia no teste:0.9821428656578064
